In [1]:
# 使用 GPU來預測 English NER
!nvidia-smi

Thu Sep  2 02:02:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    24W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install flair

     |████████████████████████████████| 319 kB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.7 MB 1.4 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 2.8 MB 46.5 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 981 kB 45.0 MB/s 
     |████████████████████████████████| 1.2 MB 30.7 MB/s 
     |████████████████████████████████| 788 kB 38.6 MB/s 
     |████████████████████████████████| 62 kB 801 kB/s 
     |████████████████████████████████| 3.3 MB 18.2 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 636 kB 52.6 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=5777a6df4e0550a340a0238dbcaabae4c7e12cc3e8a76bc58efd76948724942e
  Stored in directory: /root/.cache/p

In [4]:
import pandas as pd
import datetime
import json

def CsvSummaryHandler( enRefs , zhRefs ) :
  en_List = enRefs.split( '[NEXT]' )
  zh_List = zhRefs.split( '[NEXT]' )
  return en_List[:-1] , zh_List[:-1]

In [5]:
# 直接Load 全部會導致Memory使用超額 , Part1 Part2 Part3 依序處理資料
trainingDataFile = '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_test_v3.csv'
trainPd = pd.read_csv(trainingDataFile, sep=',', header=0)
trainPd.head()

,id,context,en_refs,zh_refs
0,<doc id=118064>,By . Sarah Griffiths . Instagram is known for ...,Art director Thomas Jullien 'to create structu...,艺术总监托马斯·朱连恩“从混乱中创造结构”，并在Instagram上把类似的图片混为一谈。[...
1,<doc id=433484>,A student claims she was accused of theft by p...,She forgot to check her change and didn't noti...,她忘了检查零钱，没有注意到燃料没有加进去。[NEXT]几小时后，警察打电话给震惊的学生，要求...
2,<doc id=618942>,A commuter train traveling in the same corrido...,A 'projectile' smashed an engineer's windshiel...,周二晚上，一枚“炮弹”打碎了北费城车站附近一辆分隔列车上一名工程师的挡风玻璃。[NEXT]没...
3,<doc id=600119>,Teenage pregnancy is a trying predicament no m...,Italian Photographer and filmmaker Paolo Patru...,意大利摄影师兼电影制作人保罗·帕特鲁诺自2011年以来一直在记录撒哈拉以南非洲和美国的产妇保...
4,<doc id=368911>,A Tory MP has caused a splash after he was spo...,Former Army captain Johnny Mercer filmed showe...,前陆军上尉约翰尼·默瑟为多芬拍摄了套子胶的广告。[NEXT]新普利茅斯议员当选前在伦敦拍摄淋...


In [6]:
print(len(trainPd))

3000


In [7]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-english-large")

# make example sentence
context = '''Art director Thomas Jullien'to create structure out of this chaos' and jumble of similar pictures on Instagram .'''
print(context)
sentence = Sentence(context)

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
  print(entity.to_dict()['labels'])
  print(type(entity.to_dict()['labels']))
  print(entity.to_dict()['labels'][0].value)

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2021-09-02 02:06:51,573 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29


Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

Art director Thomas Jullien'to create structure out of this chaos' and jumble of similar pictures on Instagram .
Sentence: "Art director Thomas Jullien'to create structure out of this chaos' and jumble of similar pictures on Instagram ."   [− Tokens: 18  − Token-Labels: "Art director Thomas <B-PER> Jullien'to <E-PER> create structure out of this chaos' and jumble of similar pictures on Instagram <S-MISC> ."]
The following NER tags are found:
[PER (1.0)]
<class 'list'>
PER
[MISC (0.5711)]
<class 'list'>
MISC


In [8]:
def GetNE_En( sentence_str ) :
  mSentence = Sentence(sentence_str)
  tagger.predict(mSentence)
  return mSentence.get_spans('ner')

In [9]:
# 抓出英文摘要中的NER 
en_refs_ne = []
for index,rowData in trainPd.iterrows() :

  context = str(rowData['en_refs'])
  context = context.replace('[NEXT]','')
  if context == 'nan' :
    print('Error資料[{}]:{}'.format(index,context))
    emptyList = []
    en_refs_ne.append(emptyList)
    continue 
  
  sum_word_nes = []
  en_sum_nes = GetNE_En(context)
  for ne in en_sum_nes :
    if ne.to_dict()['labels'][0].value == 'PER' :
      sum_word_nes.append(ne.to_dict()['text'])

  nes_set =[]
  [nes_set.append(i) for i in sum_word_nes if not i in nes_set]
  en_refs_ne.append(nes_set)

  if index % 10000 == 0 :
    print(datetime.datetime.now(),"[",index,"]")

  
# '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v3_ne_part1.csv'
# '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v3_part1.csv'
# '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v3_part1.csv'

trainPd['name_entity'] = en_refs_ne
trainPd.to_csv('/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_test_ne.csv')



2021-09-02 02:07:37.939185 [ 0 ]


In [ ]:
trainingDataFile = '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v3_part3.csv'
trainPd = pd.read_csv(trainingDataFile, sep=',', header=0)

# 抓出英文摘要中的NER 
en_refs_ne = []
for index,rowData in trainPd.iterrows() :

  context = str(rowData['en_refs'])
  context = context.replace('[NEXT]','')
  if context == 'nan' :
    print('Error資料[{}]:{}'.format(index,context))
    emptyList = []
    en_refs_ne.append(emptyList)
    continue 
  
  sum_word_nes = []
  en_sum_nes = GetNE_En(context)
  for ne in en_sum_nes :
    if ne.to_dict()['labels'][0].value == 'PER' :
      sum_word_nes.append(ne.to_dict()['text'])

  nes_set =[]
  [nes_set.append(i) for i in sum_word_nes if not i in nes_set]
  en_refs_ne.append(nes_set)

  if index % 10000 == 0 :
    print(datetime.datetime.now(),"[",index,"]")

  
# '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v3_ne_part1.csv'
# '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v3_part1.csv'
# '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v3_part1.csv'

trainPd['name_entity'] = en_refs_ne
trainPd.to_csv('/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v3_ne_part3.csv')

2021-08-24 02:11:54.908102 [ 0 ]
2021-08-24 02:17:17.781864 [ 10000 ]
2021-08-24 02:22:40.583136 [ 20000 ]
2021-08-24 02:28:05.785788 [ 30000 ]
2021-08-24 02:33:30.532018 [ 40000 ]
2021-08-24 02:38:52.386487 [ 50000 ]
2021-08-24 02:44:15.612161 [ 60000 ]
2021-08-24 02:49:41.099097 [ 70000 ]
2021-08-24 02:55:07.329724 [ 80000 ]
2021-08-24 03:00:32.889042 [ 90000 ]
2021-08-24 03:05:57.543048 [ 100000 ]
2021-08-24 03:11:22.016035 [ 110000 ]
2021-08-24 03:16:46.127356 [ 120000 ]


In [10]:
# Test
tempPd = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_test_ne.csv', sep=',', header=0)
tempPd.head()

,Unnamed: 0,id,context,en_refs,zh_refs,name_entity
0,0,<doc id=118064>,By . Sarah Griffiths . Instagram is known for ...,Art director Thomas Jullien 'to create structu...,艺术总监托马斯·朱连恩“从混乱中创造结构”，并在Instagram上把类似的图片混为一谈。[...,['Thomas Jullien']
1,1,<doc id=433484>,A student claims she was accused of theft by p...,She forgot to check her change and didn't noti...,她忘了检查零钱，没有注意到燃料没有加进去。[NEXT]几小时后，警察打电话给震惊的学生，要求...,['Ellie']
2,2,<doc id=618942>,A commuter train traveling in the same corrido...,A 'projectile' smashed an engineer's windshiel...,周二晚上，一枚“炮弹”打碎了北费城车站附近一辆分隔列车上一名工程师的挡风玻璃。[NEXT]没...,[]
3,3,<doc id=600119>,Teenage pregnancy is a trying predicament no m...,Italian Photographer and filmmaker Paolo Patru...,意大利摄影师兼电影制作人保罗·帕特鲁诺自2011年以来一直在记录撒哈拉以南非洲和美国的产妇保...,['Paolo Patruno']
4,4,<doc id=368911>,A Tory MP has caused a splash after he was spo...,Former Army captain Johnny Mercer filmed showe...,前陆军上尉约翰尼·默瑟为多芬拍摄了套子胶的广告。[NEXT]新普利茅斯议员当选前在伦敦拍摄淋...,['Johnny Mercer']


In [ ]:
print(tempPd['en_refs'][115601])
print(tempPd['name_entity'][115601])

Actors re-created the Battle of Borodino - key battle in the French-Russian war, which claimed 70,000 lives  .[NEXT]Battle in June 1812 was glorified in Leo Tolstoy's War and Peace .[NEXT]Government spent at least £700,000 on the re-enactment .[NEXT]Putin: 'Such occasions as today do not only serve to remember bloody events'[NEXT]
['Leo Tolstoy', 'Putin']


In [ ]:
def WriteCSV( fileName , id , context , en_refs , zh_refs , en_refs_nes ):
  dataframeWriter = pd.DataFrame({'id':id,'context':context,'en_refs':en_refs,'zh_refs':zh_refs,'en_refs_nes':en_refs_nes })
  dataframeWriter.to_csv( fileName, index=False, sep=',')

In [ ]:
output_id = []
output_article = []
output_enref = []
output_enref_nes = []
output_zhref = []

for index,rowData in trainPd.iterrows() :

  if str(rowData['context']) == 'nan' :
    output_id.append(index)
    output_article.append('nan')
    output_enref.append('nan')
    output_enref_nes.append('nan')
    output_zhref.append('nan')
    continue

  # 原文中的人名 - NEs
  tmp_art = rowData['context']
  tmp_sum_en , tmp_sum_zh = CsvSummaryHandler(rowData['en_refs'],rowData['zh_refs'])


  if len(tmp_art) >= 510 :
    tmp_art = tmp_art[:510]

  art_nes = GetNE_En(tmp_art)

  art_word_nes = []
  for ne in art_nes :
    if ne.to_dict()['labels'][0].value == 'PER' :
      art_word_nes.append(ne.to_dict()['text'])

  
  # sum_nes = GetNE_En(tmp_sum_en[0])
  #    sum_word_nes = []
  #  for ne in sum_nes :
  #  if ne.to_dict()['labels'][0].value == 'PER' :
  #    sum_word_nes.append(ne.to_dict()['text'])
  
  # 減少 NE 作業
  nes_set = set( art_word_nes )
  nes_sum_dict = {}
  tmp_index= 0 
  new_art = tmp_art
  for word in nes_set :
    sp_Token = '[ENNE_' + str(tmp_index) + ']'
    new_art = new_art.replace(word,sp_Token)
    nes_sum_dict[word] = tmp_index
    tmp_index += 1
  
  # Debug
  # print(new_art)
  # print(tmp_sum_en[0])
  # print(json.dumps(nes_sum_dict))
  # break

  output_id.append(index)
  output_article.append(new_art)
  output_enref.append(tmp_sum_en[0])
  output_enref_nes.append(json.dumps(nes_sum_dict))
  output_zhref.append(tmp_sum_zh[0])

  if index % 10000 == 0 :
    print(datetime.datetime.now(),"[",index,"]")


fileName = '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v2_part1_sp_name_version.csv'
WriteCSV(fileName,output_id,output_article,output_enref,output_zhref,output_enref_nes)


2021-07-07 00:37:20.714089 [ 0 ]


In [ ]:
import pandas as pd
from glob import glob
 
files = ['/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v2_part1_sp_name_version_80000.csv','/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v2_part1_sp_name_version.csv'] 
 
df = pd.concat(
    (pd.read_csv(file, usecols=['id','context','en_refs','zh_refs','en_refs_nes'], dtype={ 'id':str,'context':str,'en_refs':str,'zh_refs':str,'en_refs_nes':str}) for file in files), ignore_index=True)
print(len(df))
#df.to_csv( '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v2_part1_sp_name_version_all.csv', index=False, sep=',')

120000


In [ ]:
df.to_csv( '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v2_part1_sp_name_version_all.csv', index=False, sep=',')

In [ ]:
print(str(trainPd.iloc[82620]['context']))

if str(trainPd.iloc[82620]['context']) == 'nan' :
  print('Good') 

nan
Good


In [ ]:

print(len(output_id[0:80000]))
print(len(output_article))
print(len(output_enref))
print(len(output_enref_nes))
print(len(output_zhref))

80000
82620
82620
82620
82620


In [ ]:
# Check Point
fileName = '/content/drive/My Drive/Colab Notebooks/Bart/DataSet/EN2ZHSUM_train_v2_part1_sp_name_version_80000.csv'
WriteCSV(fileName,output_id[0:80000],output_article[0:80000],output_enref[0:80000],output_zhref[0:80000],output_enref_nes[0:80000])

In [ ]:
print(output_id[79998])
print(output_article[79998])
print(output_enref[79998])
print(output_zhref[79998])
print(output_enref_nes[79998])

79998
Celebrity accountant Anthony [ENNE_0] paid a record $11.5 million for Morning Show co-host [ENNE_2]'s clifftop mansion. The deal is easily the biggest ever for a house in the sought-after Sydney beachside suburb of Dover Heights, home to many well-heeled socialites.  . Mr [ENNE_0]'s purchase smashed the $9.5 million paid in 2014 for a similar mansion by South African tech investor [ENNE_1] and his wife [ENNE_3]. Real estate figures called it an offer 'too good to refuse', despite the eight-figure askin
Celebrity accountant Anthony Bell purchased a luxurious clifftop new home .
名人会计师安东尼·贝尔买了一套豪华的悬崖顶新房。
{"Bell": 0, "Kevin Bermeister": 1, "Larry Emdur": 2, "Beverley": 3, "Anthony Bell": 4}


In [ ]:
fileName = 'EN2ZHSUM_train_v2_part1_sp_test.csv'
WriteCSV(fileName,output_id,output_article,output_enref,output_zhref,output_enref_nes)

In [ ]:
# 1 runtime : 11:24 s
# Need 12 times
# total use : 136.8min = 2hr , 17 min

3514
676


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Bert_NER/Flair/

/content/drive/My Drive/Colab Notebooks/Bert_NER/Flair


In [ ]:
!ls

EN2ZHSUM_test_v2.csv  FlairNER.ipynb


,id,context,en_refs,zh_refs
0,<doc id=118064>,By . Sarah Griffiths . Instagram is known for ...,Art director Thomas Jullien 'to create structu...,艺术总监托马斯·朱连恩“从混乱中创造结构”，并在Instagram上把类似的图片混为一谈。[...
1,<doc id=433484>,A student claims she was accused of theft by p...,She forgot to check her change and didn't noti...,她忘了检查零钱，没有注意到燃料没有加进去。[NEXT]她忘了检查她的零钱，没有注意到燃料没有...
2,<doc id=618942>,A commuter train traveling in the same corrido...,A 'projectile' smashed an engineer's windshiel...,周二晚上，一枚“炮弹”打碎了北费城车站附近一辆分隔列车上一名工程师的挡风玻璃。[NEXT]周...
3,<doc id=600119>,Teenage pregnancy is a trying predicament no m...,Italian Photographer and filmmaker Paolo Patru...,意大利摄影师兼电影制作人保罗·帕特鲁诺自2011年以来一直在记录撒哈拉以南非洲和美国的产妇保...
4,<doc id=368911>,A Tory MP has caused a splash after he was spo...,Former Army captain Johnny Mercer filmed showe...,前陆军上尉约翰尼·默瑟为多芬拍摄了套子胶的广告。[NEXT]前陆军上尉约翰尼·默瑟拍摄了多芬...


In [ ]:
def GetNE_En( art ) :
  sentence = Sentence(art)
  tagger.predict(sentence)
  return sentence.get_spans('ner')

In [ ]:
# art
nes = GetNE_En(trainPd.loc[0]['context'][:510])
# en_sum
unitTextTemp_En , unitTextTemp_Zh = CsvSummaryHandler(trainPd.loc[0]['en_refs'],trainPd.loc[0]['zh_refs'])
sum_nes = GetNE_En(unitTextTemp_En[0])

In [ ]:
art_word_nes = []
for ne in nes : 
  art_word_nes.append(ne.to_dict()['text'])

sum_word_nes = []
for ne in sum_nes : 
  sum_word_nes.append(ne.to_dict()['text'])

In [ ]:
print(set(art_word_nes))
print(set(sum_word_nes))

{'Thomas Jullien', 'Instagram', 'Sarah Griffiths', 'Netherlands-based'}
{'Instagram', 'Thomas Jullien'}


In [ ]:
new_sum = unitTextTemp_En[0]
for index, word in enumerate(set(art_word_nes)):
  sp_Token = '[ENNER_' + str(index) + ']'
  new_sum = new_sum.replace(word,sp_Token)

print(new_sum)

Art director [ENNER_0] 'to create structure out of this chaos' and jumble of similar pictures on [ENNER_1] .


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Bart/DataSet/

/content/drive/My Drive/Colab Notebooks/Bart/DataSet


In [ ]:
!ls

data.json		     EN2ZHSUM_train_Part_3.csv
datav1.json		     EN2ZHSUM_train.txt
datav2.json		     EN2ZHSUM_train_v2.csv
datav3.json		     EN2ZHSUM_train_v2_part1.csv
EN2ZHSUM_test.txt	     EN2ZHSUM_train_v2_part2.csv
EN2ZHSUM_test_v2.csv	     EN2ZHSUM_train_v2_part3.csv
EN2ZHSUM_train_for_ner.json  en_zh_dict.txt
EN2ZHSUM_train_Part_1.csv    trained_summary_fzpar_model_v3
EN2ZHSUM_train_Part_2.csv    trained_summary__testv1_model


In [ ]:
trainingDataFile = 'EN2ZHSUM_train_v2_part1.csv'
trainPd = pd.read_csv(trainingDataFile, sep=',', header=0)
trainPd.head()

,id,context,en_refs,zh_refs
0,<doc id=159800>,"By . John Stevens . PUBLISHED: . 07:19 EST, 12...",Rod Gray was taken to Ipswich Hospital after f...,罗德·格雷在家中摔倒后被送往伊普斯维奇医院。[NEXT]他的家人声称他独自一人，需要厕所，陷...
1,<doc id=123790>,Jordan Rossiter grabbed the winner as Liverpoo...,Jordan Rossiter scored a second-half winner fo...,乔丹·罗塞特下半场在朗特里公园为利物浦攻入一球。[NEXT]这位中场球员本赛季已经入选了首都...
2,<doc id=385119>,Divisions between Turkey and Europe have inten...,The Prime Minister today launched an extraordi...,首相今天对欧盟领导人发起了一次非同寻常的攻击。[NEXT]雷杰普·塔伊普·埃尔多安正在应对移...
3,<doc id=88967>,An ornate scabbard chape belonging to Admiral ...,It was discovered by a police diver on the bed...,1970年，一名警察潜水员在泰晤士河河床上发现了它。[NEXT]霍雷肖·纳尔逊在带领他的士兵...
4,<doc id=326245>,A man has been charged with animal cruelty aft...,A member of the public saw a 36-year-old man a...,一名公众在彭里斯看到一名36岁的男子攻击他的狗。[NEXT]尽管狗大声狂吠，那人还是继续拳打...


In [ ]:
# Func
def WriteCSV( fileName , id , context , en_refs , zh_refs , en_refs_nes ):
  dataframeWriter = pd.DataFrame({'id':id,'context':context,'en_refs':en_refs,'zh_refs':zh_refs,'en_refs_nes':en_refs_nes })
  dataframeWriter.to_csv( fileName, index=False, sep=',')

In [ ]:
import datetime

In [ ]:
output_id = []
output_article = []
output_enref = []
output_enref_nes = []
output_zhref = []


for index,rowData in trainPd.iterrows() :
  #tmp_art = rowData['context']
  tmp_sum_en , tmp_sum_zh = CsvSummaryHandler(rowData['en_refs'],rowData['zh_refs'])

  #if len(tmp_art) >= 510 :
  #  tmp_art = tmp_art[:510]

  #art_nes = GetNE_En(tmp_art)
  sum_nes = GetNE_En(tmp_sum_en[0])

  #art_word_nes = []
  sum_word_nes = []
  #for ne in art_nes : 
  #  art_word_nes.append(ne.to_dict()['text'])
  for ne in sum_nes : 
    sum_word_nes.append(ne.to_dict()['text'])
  #art_word_nes = set(art_word_nes)
  #sum_word_nes = set(sum_word_nes)
  
  #new_art = rowData['context']
  new_sum = tmp_sum_en[0]
  tmp_index = 0

  # for word in art_word_nes :
  # sp_Token = '[ENNER_' + str(tmp_index) + ']'
  # new_art = new_art.replace(word,sp_Token)
  # new_sum = new_sum.replace(word,sp_Token)
  # tmp_index += 1

  for word in sum_word_nes :
    sp_Token = '[ENNER_' + str(tmp_index) + ']'
    new_sum = new_sum.replace(word,sp_Token)
    tmp_index += 1
  
  if index % 10000 == 0 :
    print(datetime.datetime.now(),"[",index,"]")

  output_id.append(index)
  #output_article.append(new_art)
  output_article.append(rowData['context'])
  output_enref.append(new_sum)
  output_enref_nes.append(('[SEP]'.join(sum_word_nes)))
  output_zhref.append(tmp_sum_zh[0])

WriteCSV('EN2ZHSUM_train_v2_part1_sp.csv',output_id,output_article,output_enref,output_zhref,output_enref_nes)

2021-06-30 08:06:35.160016 [ 0 ]


In [ ]:
mJsonObect = []
for index in range(3000) : 
  tmpJsonData = { 'id':output_id[index], 'text':output_zhref[index] }
  mJsonObect.append(tmpJsonData)

with open("EN2ZHSUM_text_for_ner.json","w",encoding='utf-8') as f:
    json.dump(mJsonObect,f,ensure_ascii=False);

In [ ]:
print(len(output_id))
print(len(output_zhref))


3000
3000


In [ ]:
mJsonObect = []
for index,rowData in trainPd.iterrows() :
  unitTextTemp_En , unitTextTemp_Zh = CsvSummaryHandler(rowData['en_refs'],rowData['zh_refs'])
  tmpJsonData = { 'id':index, 'text':unitTextTemp_Zh[0] }
  mJsonObect.append(tmpJsonData)
  
with open("EN2ZHSUM_train_for_ner.json","w",encoding='utf-8') as f:
    json.dump(mJsonObect,f,ensure_ascii=False);